# Author: Olufemi Victor.

# Title: Multiple Colour Detection.

Following a tutorial I previously made on Tracking my red pen with Opencv, i decided to upgrade the tutorial to suit a task at my internship with The Sparks Foundation. The task involves multiple color detection.

I explained the procedures quite well, but feel free to check out my previous tutorials on opencv in my [github repo](github.com/osinkolu).

###### The Pipeline
The pipeline is quite easy, First is to get the frame, mask out all other color regions asides our color of interest, and lastly draw a bounding circle for the contour. I have gone through the pain to track the specific hsv boundaries for the provided colors

###### Import Packages.
Firstly, we import the packages needed to run the algorithm. If you've been following my previous tutorials.

Time is another package available in python which helps us keep track of real time.

In [64]:
from collections import deque # import deque a far faster list
import numpy as np # import numpy as needed
import imutils # we definately need imutils convienience functions.
import time # time is time
import cv2 # The major library, opencv
import pandas as pd #get victor's csv file of colour boundary.

In [65]:
# import my hsv specifications
victors_hsv_specs = pd.read_csv('victors_hsv_specs.csv')

The contents of the csv file include the colour name which every column has, the first row with index zero is the lower hsv boundary, the second row is the upper hsv boundary, and the third row is the BGR color code for the specified color.

In [66]:
# view the csv contents.
victors_hsv_specs.head()

,white,black,red,blue,yellow,leamon,green,pink,brown,purple,gold
0,"(0, 0, 180)","(0, 0, 0)","(0, 150, 150)","(100, 100, 50)","(20, 100, 100)","(30, 100, 100)","(70, 110, 110)","(160, 100, 100)","(260, 100, 100)","(122, 100, 100)","(260, 100, 80)"
1,"(180, 20, 255)","(180, 255, 20)","(5, 255, 225)","(120, 255, 255)","(30, 255, 225)","(50, 255, 128)","(90, 255, 255)","(170, 255, 255)","(270, 255, 255)","(160, 255, 255)","(280, 200, 160)"
2,"(255, 255, 255)","(0, 0, 0)","(0, 0, 255)","(255, 0, 0)","(0, 255, 255)","(0, 255, 0)","(0, 128, 0)","(147, 20, 255)","(139, 69, 19)","(128, 0, 128)","(0, 215, 255)"


In [67]:
# since data is vry small, for faster and easier maipulation, i converted it to a dictionary format.
victors_hsv_specs = victors_hsv_specs.to_dict()

In [68]:
# see white upper boundary
eval(victors_hsv_specs["white"][1])

(180, 20, 255)

Finally i start the video Stream giving it some 2 seconds to warm up.

In [69]:
print("Booting the Video stream,")
vs = cv2.VideoCapture("many_things.mp4")# uncomment this to load in your own video stream.
#vs = cv2.VideoCapture(0)# uncomment this if you want to stream in real time.
time.sleep(2.0) #set sleep time to 2.0 seconds

Booting the Video stream,


###### The Big Deal
in this section, we do everything. First, i placed the whole process in a while loop since a video is just continous moving frame of pictures.

In the next line i read off the frame from the video master, next, i seperate the ret and frame. The ret is simply just a boolean regarding whether there was a return at all from the vs.read(). Next if there is nothing in the frame, break stop the process.Another way could be if ret is false, break. In the next line, i used imutils to resize the frame. The imutils resize saves a lot of stress in preserving the aspect ratio than stressful standard OpenCV methods.

Following that, i decided to flip the image. I did this to obtain the mirror view of my frame which is absolutely quite cool for direct streaming from the camera.

###### Processing The Frame.
The frame is ready, first i blur the frame using the popular gaussian blur method, this helps to reduce noise in the image. Next, i convert the colour from the BGR to HSV. The HSV is a colour band which means Hue, Saturation and Value. it is a colour representation closer to the Human Vision(our way of seeing). 

The next step is where my for loop comes in. Each time my computer looks at a video frame, i looks at it Ten times, because i have 10 colors in my dictionary, for each time it looks at the picture, it checks for a specific colour in the image and labels it. How is this done??

to achieve this in the next step, what i did was to create a mask by thresholding the image such that i save out only colours within my upper and lower boundary, which in this case is the colour of my red pen. After masking out other parts of the image, i erode, and dilate to futher reduce high frequency noise caused by thresholding.

###### All said and done??
Now that we've gotten a view of the objects(based on the color), we find contours, grab them using imutils. In the following IF statement, i pick out the contour with the maximum Area,(.....Understand that you should very much edit this part to your own taste, the reason for this was to prevent wrong color detection due to CRAZY high frequency noise that escapes dilation, erosion......) this maximum contour is definately the contour of my object of interest, the remaining contours may or may not be noise. In the next line, i prepare parameters required to construct a bounding circle on the object. The first is the radius and the xy co-ordinates, next i used cv2.Moments to obtain it's center. Then if the radius is reasonable enough, i draw a bounding circle around my object with the detected colour as well as the name of the colour out with the detected colour, cool right?

In [70]:
while True:
    frame = vs.read() #Read off the frame from the video stream
    ret,frame  = frame # Use this if you want to load in your video
    if frame is None: # If there is no frame, save my pc from going through any stress at all
        break
    # otherwise, if we have a frame, we proceed with the following code
    frame = imutils.resize(frame, width = 700) # so much easier than open cv, keeping aspect ratio intact
    frame = cv2.flip(frame,1) # i want the mirror view, it's very helpful especially if i'm streaming
    
    #processing the frame 
    blurred = cv2.GaussianBlur(frame, (11,11),0) # blurr helps to reduce high frequency noise, definately helps model
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV) # convert my color to the HSV format
    
    for i in victors_hsv_specs.keys(): # check image multiple times for a different colour
        lower_color_boundary = eval(victors_hsv_specs[i][0]) # assign lower boundary
        upper_color_boundary = eval(victors_hsv_specs[i][1]) # assign Upper boundary
        BGR_SPECTRUM = eval(victors_hsv_specs[i][2]) # assign BGR Spectrum to write names.
    
        # Create a mask
        mask = cv2.inRange(hsv, lower_color_boundary, upper_color_boundary) # mask other regions except colors in range of upper to lower (thresholding)
        mask = cv2.erode(mask, None, iterations =2) # Reduce noise caused by thresholding
        mask = cv2.dilate(mask, None, iterations =2) # foreground the found object i.e futher reduce noise.


        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find contours
        contours = imutils.grab_contours(contours) # Grab the contours using imutils
        if len(contours) > 0: # if the contours list is not empty proceed
            contour = max(contours, key = cv2.contourArea) # select contour with maximum Area, most likely our object
            ((x,y), radius) = cv2.minEnclosingCircle(contour) # pick up co-ordinates for drawing a circle around the object
            if radius > 10: # if we have a reasonable radius for the proposed object detected
                cv2.circle(frame, (int(x), int(y)), int(radius), BGR_SPECTRUM, 2) # Draw a circle to bound the Object
                cv2.putText(frame, i, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.7,BGR_SPECTRUM, 2) # write names
            cv2.imshow("Frame by Frame makes Video", frame) # let's see the frame X frame
            
    # Closing a video frame
    key = cv2.waitKey(1) #wait for the cv key
    if key == ord("x"): # If the x button is pressed
        break # Break from the loop
vs.release() # Let opencv release the video loader
cv2.destroyAllWindows() # Destroy all windows to close it

In my quest to manually find the HSV values for these colors as embedded in my hsv file, i stumbled on a tutorial post by opencv, as the creators, they implemented already a simpler version of the code i've just written, they confirmed that the quest to find true boundaries for each colour was quite tedious, they suggested a far easier way to get the hsv colour code. check out the post [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html). Note that my hsv values range do not totally correlate with the post, but it helped quite a lot. Make sure to check it out and improve the victors_hsv_specs CSV file. Feel free to reach out to me on linkedIn [here](https://www.linkedin.com/in/olufemi-victor-tolulope/)

In [8]:
# what the post entails.................
colour_in_bgr = np.uint8([[[139,69,19 ]]])# ...........slot in colour in BGR format
hsv_ = cv2.cvtColor(colour_in_bgr,cv2.COLOR_BGR2HSV) #Convert to hsv
print (hsv_)# print out HSV color code.

[[[108 220 139]]]
